In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("POWER BI DATA PREPARATION - FINAL ASSEMBLY")
print("=" * 60)

POWER BI DATA PREPARATION - FINAL ASSEMBLY


In [2]:
# 1. LOAD ALL DATA SOURCES
print("\n1. LOADING DATA SOURCES...")

try:
    # Load original datasets
    train_df = pd.read_csv('../data/raw/train.csv')
    store_df = pd.read_csv('../data/raw/store.csv')
    test_df = pd.read_csv('../data/raw/test.csv')
    
    # Load processed data
    processed_df = pd.read_csv('../data/processed/rossmann_processed.csv')
    processed_df['Date'] = pd.to_datetime(processed_df['Date'])
    
    # Load forecast results
    forecast_df = pd.read_csv('../data/forecasts/prophet_forecast_powerbi.csv')
    forecast_df['Date'] = pd.to_datetime(forecast_df['Date'])
    
    # Load model metrics
    metrics_df = pd.read_csv('../data/forecasts/model_metrics.csv')
    
    print(f"Train data: {train_df.shape}")
    print(f"Store data: {store_df.shape}")
    print(f"Test data: {test_df.shape}")
    print(f"Processed data: {processed_df.shape}")
    print(f"Forecast data: {forecast_df.shape}")
    print(f"Metrics data: {metrics_df.shape}")
    
except FileNotFoundError as e:
    print(f"Missing file: {e}")
    print("Please ensure you've run 01_EDA.ipynb then 02_Modeling.ipynb first")
    raise


1. LOADING DATA SOURCES...
Train data: (1017209, 9)
Store data: (1115, 10)
Test data: (41088, 8)
Processed data: (844392, 26)
Forecast data: (942, 12)
Metrics data: (3, 3)


In [3]:
# 2. CREATE MASTER DATASET FOR POWER BI
print("\n2. CREATING MASTER DATASET...")

# Start with forecast data as base (includes historical + future)
master_df = forecast_df.copy()

# Add detailed historical data where available
historical_detail = processed_df.groupby('Date').agg({
    'Sales': 'sum',
    'Customers': 'sum',
    'Open': 'mean',
    'Promo': 'mean',
    'StateHoliday': lambda x: (x != '0').mean(),
    'SchoolHoliday': 'mean',
    'Store': 'nunique',
    'StoreType': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'a',
    'Assortment': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'a',
    'CompetitionDistance': 'mean',
    'DayOfWeek': 'first',
    'IsWeekend': 'mean',
    'Year': 'first',
    'Month': 'first'
}).reset_index()

# Merge with master dataset
master_df = master_df.merge(historical_detail, on='Date', how='left', suffixes=('', '_detail'))

# Clean up column names and add missing features
master_df['Total_Stores_Open'] = master_df['Store'].fillna(1115)  # Total stores in dataset
master_df['Avg_Customers_Per_Store'] = master_df['Customers'] / master_df['Total_Stores_Open']
master_df['Sales_Per_Customer'] = master_df['Actual_Sales'] / master_df['Customers']
master_df['Promo_Penetration'] = master_df['Promo'].fillna(0.4)  # Use recent average for future
master_df['Holiday_Penetration'] = master_df['StateHoliday'].fillna(0.05)

print(f"Master dataset created: {master_df.shape}")


2. CREATING MASTER DATASET...
Master dataset created: (942, 31)


In [4]:
# 3. CREATE STORE-LEVEL AGGREGATIONS
print("\n3. CREATING STORE-LEVEL SUMMARIES...")

# Store performance summary
store_summary = processed_df.groupby('Store').agg({
    'Sales': ['sum', 'mean', 'std'],
    'Customers': ['sum', 'mean'],
    'StoreType': 'first',
    'Assortment': 'first',
    'CompetitionDistance': 'first'
}).round(2)

# Flatten column names
store_summary.columns = ['_'.join(col).strip() for col in store_summary.columns.values]
store_summary = store_summary.reset_index()

# Add performance categories
store_summary['Sales_Category'] = pd.cut(
    store_summary['Sales_mean'], 
    bins=[0, 5000, 7000, 10000, float('inf')],
    labels=['Low', 'Medium', 'High', 'Very High']
)

print(f"Store summary created: {store_summary.shape}")



3. CREATING STORE-LEVEL SUMMARIES...
Store summary created: (1115, 10)


In [5]:
# 4. CREATE TIME-BASED AGGREGATIONS
print("\n4. CREATING TIME-BASED SUMMARIES...")

# Monthly aggregations
monthly_summary = processed_df.groupby(['Year', 'Month']).agg({
    'Sales': ['sum', 'mean'],
    'Customers': ['sum', 'mean'],
    'Promo': 'mean',
    'StateHoliday': lambda x: (x != '0').mean(),
    'Store': 'nunique'
}).round(2)

monthly_summary.columns = ['_'.join(col).strip() for col in monthly_summary.columns.values]
monthly_summary = monthly_summary.reset_index()
monthly_summary['Month_Year'] = monthly_summary['Year'].astype(str) + '-' + monthly_summary['Month'].astype(str).str.zfill(2)

# Weekly aggregations  
weekly_summary = processed_df.groupby(['Year', processed_df['Date'].dt.isocalendar().week]).agg({
    'Sales': ['sum', 'mean'],
    'Customers': ['sum', 'mean'],
    'Promo': 'mean'
}).round(2)

weekly_summary.columns = ['_'.join(col).strip() for col in weekly_summary.columns.values]
weekly_summary = weekly_summary.reset_index()
weekly_summary.columns = ['Year', 'Week', 'Sales_sum', 'Sales_mean', 'Customers_sum', 'Customers_mean', 'Promo_mean']

# Day of week patterns
dow_summary = processed_df.groupby('DayOfWeek').agg({
    'Sales': ['sum', 'mean', 'std'],
    'Customers': ['sum', 'mean']
}).round(2)

dow_summary.columns = ['_'.join(col).strip() for col in dow_summary.columns.values]
dow_summary = dow_summary.reset_index()
dow_summary['Day_Name'] = dow_summary['DayOfWeek'].map({
    0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday',
    4: 'Friday', 5: 'Saturday', 6: 'Sunday'
})

print(f"Monthly summary: {monthly_summary.shape}")
print(f"Weekly summary: {weekly_summary.shape}")
print(f"Day of week summary: {dow_summary.shape}")



4. CREATING TIME-BASED SUMMARIES...
Monthly summary: (31, 10)
Weekly summary: (135, 7)
Day of week summary: (7, 7)


In [6]:
# 5. CREATE KPI SUMMARY TABLE
print("\n5. CREATING KPI SUMMARY...")

# Calculate key business metrics
total_historical_sales = processed_df['Sales'].sum()
total_historical_customers = processed_df['Customers'].sum()
avg_daily_sales = processed_df.groupby('Date')['Sales'].sum().mean()
best_performing_store = processed_df.groupby('Store')['Sales'].mean().idxmax()
best_store_avg_sales = processed_df.groupby('Store')['Sales'].mean().max()

# Future forecasts summary
future_forecasts = master_df[master_df['Is_Forecast'] == True]
if len(future_forecasts) > 0:
    forecasted_total_sales = future_forecasts['Forecasted_Sales'].sum()
    avg_forecasted_daily_sales = future_forecasts['Forecasted_Sales'].mean()
    forecast_period_days = len(future_forecasts)
else:
    forecasted_total_sales = 0
    avg_forecasted_daily_sales = 0
    forecast_period_days = 0

# Create KPI dataframe
kpi_summary = pd.DataFrame({
    'KPI_Name': [
        'Total Historical Sales',
        'Total Historical Customers', 
        'Average Daily Sales (Historical)',
        'Best Performing Store ID',
        'Best Store Average Daily Sales',
        'Forecasted Total Sales',
        'Average Forecasted Daily Sales',
        'Forecast Period (Days)',
        'Model MAPE',
        'Model MAE',
        'Model RMSE'
    ],
    'Value': [
        f"€{total_historical_sales:,.0f}",
        f"{total_historical_customers:,.0f}",
        f"€{avg_daily_sales:,.0f}", 
        f"{best_performing_store}",
        f"€{best_store_avg_sales:,.0f}",
        f"€{forecasted_total_sales:,.0f}",
        f"€{avg_forecasted_daily_sales:,.0f}",
        f"{forecast_period_days}",
        f"{metrics_df[metrics_df['Metric'] == 'MAPE']['Value'].iloc[0]:.1f}%",
        f"€{metrics_df[metrics_df['Metric'] == 'MAE']['Value'].iloc[0]:,.0f}",
        f"€{metrics_df[metrics_df['Metric'] == 'RMSE']['Value'].iloc[0]:,.0f}"
    ],
    'Category': [
        'Historical Performance',
        'Historical Performance',
        'Historical Performance', 
        'Store Performance',
        'Store Performance',
        'Future Projections',
        'Future Projections',
        'Future Projections',
        'Model Performance',
        'Model Performance', 
        'Model Performance'
    ]
})

print(f"KPI summary created: {kpi_summary.shape}")


5. CREATING KPI SUMMARY...
KPI summary created: (11, 3)


In [7]:
# 6. CREATE SEASONAL INSIGHTS TABLE
print("\n6. CREATING SEASONAL INSIGHTS...")

# Monthly seasonality
monthly_seasonality = processed_df.groupby('Month')['Sales'].mean().reset_index()
monthly_seasonality['Month_Name'] = monthly_seasonality['Month'].map({
    1: 'January', 2: 'February', 3: 'March', 4: 'April',
    5: 'May', 6: 'June', 7: 'July', 8: 'August', 
    9: 'September', 10: 'October', 11: 'November', 12: 'December'
})
monthly_seasonality['Seasonality_Type'] = 'Monthly'

# Day of week seasonality  
dow_seasonality = dow_summary[['DayOfWeek', 'Day_Name', 'Sales_mean']].copy()
dow_seasonality.columns = ['Period_Number', 'Period_Name', 'Sales']
dow_seasonality['Seasonality_Type'] = 'Weekly'

# Combine seasonality data
seasonality_insights = pd.concat([
    monthly_seasonality[['Month', 'Month_Name', 'Sales']].rename(columns={'Month': 'Period_Number', 'Month_Name': 'Period_Name'}),
    dow_seasonality
], ignore_index=True)

print(f"Seasonality insights: {seasonality_insights.shape}")


6. CREATING SEASONAL INSIGHTS...
Seasonality insights: (19, 4)


In [8]:
# 7. DATA QUALITY CHECKS
print("\n7. PERFORMING DATA QUALITY CHECKS...")

quality_checks = {
    'Master Dataset': {
        'Total Rows': len(master_df),
        'Date Range': f"{master_df['Date'].min()} to {master_df['Date'].max()}",
        'Missing Actual Sales': master_df['Actual_Sales'].isna().sum(),
        'Missing Forecasted Sales': master_df['Forecasted_Sales'].isna().sum(),
        'Future Records': (master_df['Is_Forecast'] == True).sum()
    },
    'Store Summary': {
        'Total Stores': len(store_summary),
        'Missing Store Types': store_summary['StoreType_first'].isna().sum(),
        'Sales Categories': store_summary['Sales_Category'].value_counts().to_dict()
    }
}

print("DATA QUALITY REPORT:")
for dataset, checks in quality_checks.items():
    print(f"\n{dataset}:")
    for check, value in checks.items():
        print(f"   • {check}: {value}")


7. PERFORMING DATA QUALITY CHECKS...
DATA QUALITY REPORT:

Master Dataset:
   • Total Rows: 942
   • Date Range: 2013-01-01 00:00:00 to 2015-07-31 00:00:00
   • Missing Actual Sales: 0
   • Missing Forecasted Sales: 0
   • Future Records: 0

Store Summary:
   • Total Stores: 1115
   • Missing Store Types: 0
   • Sales Categories: {'Medium': 440, 'High': 374, 'Low': 207, 'Very High': 94}


In [9]:
# 8. SAVE ALL POWER BI DATASETS
print("\n8. SAVING POWER BI DATASETS...")

# Create forecasts folder if it doesn't exist
import os
os.makedirs('../data/forecasts', exist_ok=True)

# Save main datasets
datasets = {
    'master_data_powerbi.csv': master_df,
    'store_summary_powerbi.csv': store_summary, 
    'monthly_summary_powerbi.csv': monthly_summary,
    'weekly_summary_powerbi.csv': weekly_summary,
    'day_of_week_summary_powerbi.csv': dow_summary,
    'kpi_summary_powerbi.csv': kpi_summary,
    'seasonality_insights_powerbi.csv': seasonality_insights
}

print("SAVING DATASETS:")
for filename, dataset in datasets.items():
    filepath = f'../data/forecasts/{filename}'
    dataset.to_csv(filepath, index=False)
    print(f"   {filename}: {dataset.shape}")


8. SAVING POWER BI DATASETS...
SAVING DATASETS:
   master_data_powerbi.csv: (942, 31)
   store_summary_powerbi.csv: (1115, 10)
   monthly_summary_powerbi.csv: (31, 10)
   weekly_summary_powerbi.csv: (135, 7)
   day_of_week_summary_powerbi.csv: (7, 7)
   kpi_summary_powerbi.csv: (11, 3)
   seasonality_insights_powerbi.csv: (19, 4)


In [10]:
# 9. CREATE DATA DICTIONARY
print("\n9. CREATING DATA DICTIONARY...")

data_dictionary = pd.DataFrame({
    'Dataset': [
        'master_data_powerbi.csv', 'master_data_powerbi.csv', 'master_data_powerbi.csv', 
        'master_data_powerbi.csv', 'master_data_powerbi.csv', 'master_data_powerbi.csv',
        'store_summary_powerbi.csv', 'store_summary_powerbi.csv', 'store_summary_powerbi.csv',
        'kpi_summary_powerbi.csv', 'seasonality_insights_powerbi.csv'
    ],
    'Column_Name': [
        'Date', 'Actual_Sales', 'Forecasted_Sales', 'Lower_Bound', 'Upper_Bound', 'Is_Forecast',
        'Store', 'Sales_mean', 'Sales_Category', 'KPI_Name', 'Seasonality_Type'
    ],
    'Description': [
        'Date of sales record',
        'Actual historical sales in euros',
        'Prophet model forecasted sales in euros', 
        'Lower confidence bound for forecast',
        'Upper confidence bound for forecast',
        'Boolean flag: True for forecasted records',
        'Store ID number',
        'Average daily sales per store',
        'Store performance category (Low/Medium/High/Very High)',
        'Key performance indicator name',
        'Type of seasonality (Monthly/Weekly)'
    ],
    'Data_Type': [
        'Date', 'Numeric', 'Numeric', 'Numeric', 'Numeric', 'Boolean',
        'Numeric', 'Numeric', 'Categorical', 'Text', 'Categorical'  
    ]
})

data_dictionary.to_csv('../data/forecasts/data_dictionary.csv', index=False)
print(f"Data dictionary saved: {data_dictionary.shape}")


9. CREATING DATA DICTIONARY...
Data dictionary saved: (11, 4)


In [12]:
# 10. CREATE POWER BI IMPORT GUIDE
print("\n10. CREATING POWER BI IMPORT GUIDE...")

powerbi_guide = """
# POWER BI IMPORT GUIDE - ROSSMANN SALES FORECASTING

## DATASETS TO IMPORT:

### 1. MAIN FORECAST DATA
**File:** master_data_powerbi.csv
**Purpose:** Primary dataset with historical sales + forecasts
**Key Columns:** Date, Actual_Sales, Forecasted_Sales, Is_Forecast
**Relationships:** Primary table for time-based analysis

### 2. STORE PERFORMANCE 
**File:** store_summary_powerbi.csv  
**Purpose:** Store-level performance metrics
**Key Columns:** Store, Sales_mean, Sales_Category, StoreType_first
**Relationships:** Link to master data via Store ID

### 3. TIME AGGREGATIONS
**Files:** monthly_summary_powerbi.csv, weekly_summary_powerbi.csv, day_of_week_summary_powerbi.csv
**Purpose:** Pre-calculated time-based summaries for performance
**Key Columns:** Various time periods with sales summaries

### 4. KPI DASHBOARD
**File:** kpi_summary_powerbi.csv
**Purpose:** Key business metrics for dashboard cards
**Key Columns:** KPI_Name, Value, Category

### 5. SEASONALITY ANALYSIS
**File:** seasonality_insights_powerbi.csv
**Purpose:** Monthly and weekly seasonal patterns  
**Key Columns:** Period_Name, Sales, Seasonality_Type

## SUGGESTED DASHBOARD PAGES:

### PAGE 1: EXECUTIVE SUMMARY
- KPI cards (Total Sales, Forecast Accuracy, etc.)
- Sales trend line (Actual vs Forecasted)
- YoY growth metrics
- Top performing stores

### PAGE 2: SALES FORECASTING
- Interactive forecast chart with confidence intervals
- Filter by date range
- Forecast vs actual comparison
- Seasonal decomposition

### PAGE 3: STORE PERFORMANCE
- Store performance matrix
- Geographic analysis (if coordinates available)
- Store type comparisons
- Bottom/Top performers

### PAGE 4: SEASONAL INSIGHTS  
- Monthly seasonality patterns
- Day-of-week analysis
- Holiday impact analysis
- Promotion effectiveness

## POWER BI SETUP STEPS:

1. **Import Data:**
   - Get Data > Text/CSV
   - Import all CSV files from /data/forecasts/
   - Ensure Date columns are recognized as Date type

2. **Create Relationships:**
   - Link master_data to store_summary via Store column
   - Set up date table relationships

3. **Create Measures:**
   - Sales Growth % = (Current Period Sales - Previous Period Sales) / Previous Period Sales
   - Forecast Accuracy = 1 - ABS(Actual - Forecast) / Actual
   - YTD Sales = TOTALYTD(SUM(Sales), Date)

4. **Add Filters:**
   - Date range slicer
   - Store type filter
   - Is_Forecast toggle

## VISUALIZATION RECOMMENDATIONS:

- **Line Charts:** Time series with dual axis for actual vs forecast
- **Cards:** KPI values with trend indicators
- **Bar Charts:** Store performance rankings
- **Heat Maps:** Seasonal patterns by month/day
- **Scatter Plots:** Forecast accuracy analysis

## DESIGN TIPS:

- Use consistent color scheme (blue for actual, orange for forecast)
- Add conditional formatting for performance categories
- Include tooltips with additional context
- Use bookmarks for different view states
- Add export functionality for reports

## ADVANCED FEATURES:

- **What-If Parameters:** Adjust promotion/holiday impacts
- **Drill-Through Pages:** Store-level detailed analysis
- **Custom Visuals:** Prophet forecast decomposition
- **Alerts:** Set up data-driven alerts for significant changes

## DATA REFRESH:

- Set up scheduled refresh if connecting to live data source
- Update forecast models monthly for best accuracy
- Monitor data quality with automated checks

---
Generated by Rossmann Sales Forecasting Pipeline
Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}
"""

with open('../data/forecasts/PowerBI_Import_Guide.md', 'w') as f:
    f.write(powerbi_guide)

print("Power BI import guide created")


10. CREATING POWER BI IMPORT GUIDE...
Power BI import guide created


In [13]:
# 11. FINAL SUMMARY AND VALIDATION
print("\n11. FINAL VALIDATION...")

# Validate data continuity
date_gaps = pd.date_range(start=master_df['Date'].min(), 
                         end=master_df['Date'].max(), 
                         freq='D').difference(master_df['Date'])

print("FINAL VALIDATION RESULTS:")
print(f"   • Total datasets created: {len(datasets)}")
print(f"   • Date range coverage: {master_df['Date'].min()} to {master_df['Date'].max()}")
print(f"   • Missing dates in master data: {len(date_gaps)}")
print(f"   • Historical records: {(master_df['Is_Forecast'] == False).sum()}")
print(f"   • Forecast records: {(master_df['Is_Forecast'] == True).sum()}")
print(f"   • Stores covered: {processed_df['Store'].nunique()}")

# Create final file manifest
file_manifest = pd.DataFrame({
    'Filename': list(datasets.keys()) + ['data_dictionary.csv', 'PowerBI_Import_Guide.md'],
    'Purpose': [
        'Main forecast dataset with historical + predicted sales',
        'Store-level performance summary and categories', 
        'Monthly aggregated sales and metrics',
        'Weekly aggregated sales and trends',
        'Day-of-week patterns and seasonality',
        'Key performance indicators for dashboard cards',
        'Seasonal insights for monthly/weekly patterns',
        'Data dictionary explaining all columns',
        'Step-by-step Power BI setup guide'
    ],
    'Record_Count': [
        len(master_df), len(store_summary), len(monthly_summary), 
        len(weekly_summary), len(dow_summary), len(kpi_summary),
        len(seasonality_insights), len(data_dictionary), 'N/A'
    ]
})

file_manifest.to_csv('../data/forecasts/file_manifest.csv', index=False)
print(f"File manifest created: {file_manifest.shape}")

print("\nPOWER BI DATA PREPARATION COMPLETE!")
print("=" * 60)
print("ALL FILES READY FOR POWER BI:")
print("   /data/forecasts/ contains all CSV files")
print("   PowerBI_Import_Guide.md has setup instructions")
print("   data_dictionary.csv explains all columns")
print("   file_manifest.csv lists all deliverables")

print(f"\n NEXT STEPS:")
print("   1. Open Power BI Desktop")
print("   2. Import CSV files from ../data/forecasts/")
print("   3. Follow the PowerBI_Import_Guide.md instructions")
print("   4. Build your interactive forecasting dashboard!")
print("   5. Share insights with business stakeholders")

print("\n PROJECT COMPLETE - READY FOR BUSINESS IMPACT!")
print("=" * 60)


11. FINAL VALIDATION...
FINAL VALIDATION RESULTS:
   • Total datasets created: 7
   • Date range coverage: 2013-01-01 00:00:00 to 2015-07-31 00:00:00
   • Missing dates in master data: 0
   • Historical records: 942
   • Forecast records: 0
   • Stores covered: 1115
File manifest created: (9, 3)

POWER BI DATA PREPARATION COMPLETE!
ALL FILES READY FOR POWER BI:
   /data/forecasts/ contains all CSV files
   PowerBI_Import_Guide.md has setup instructions
   data_dictionary.csv explains all columns
   file_manifest.csv lists all deliverables

 NEXT STEPS:
   1. Open Power BI Desktop
   2. Import CSV files from ../data/forecasts/
   3. Follow the PowerBI_Import_Guide.md instructions
   4. Build your interactive forecasting dashboard!
   5. Share insights with business stakeholders

 PROJECT COMPLETE - READY FOR BUSINESS IMPACT!
